In [335]:
import pandas as pd
import numpy as np
import collections

from datetime import datetime, timedelta 

import matplotlib.pyplot as plt
import plotly.express as px
import plotly.graph_objs as go
from plotly.subplots import make_subplots

from pycaret.regression import *
from sklearn.model_selection import TimeSeriesSplit
from sklearn.preprocessing import LabelEncoder
import xgboost

In [336]:
train = pd.read_csv('../input/train.csv')
train.head()

,id,accommodates,amenities,bathrooms,bed_type,bedrooms,beds,cancellation_policy,city,cleaning_fee,...,longitude,name,neighbourhood,number_of_reviews,property_type,review_scores_rating,room_type,thumbnail_url,zipcode,y
0,0,6,"{TV,""Wireless Internet"",Kitchen,""Free parking ...",2.0,Real Bed,1.0,4.0,flexible,LA,t,...,-118.154761,The Penthouse,NaN,1,Apartment,60.0,Private room,NaN,90804,138.0
1,1,2,"{TV,""Cable TV"",Internet,""Wireless Internet"",""A...",1.0,Real Bed,1.0,1.0,strict,DC,t,...,-76.978190,Guest Bedroom in Brookland,Brookland,9,House,100.0,Private room,https://a0.muscache.com/im/pictures/e4d8b51f-6...,20018,42.0
2,2,2,"{TV,Internet,""Wireless Internet"",Kitchen,""Indo...",2.0,Real Bed,1.0,1.0,strict,NYC,t,...,-73.926240,Clean Modern Room in Lux Apt 1 Block From J Train,Bushwick,27,Apartment,83.0,Private room,https://a0.muscache.com/im/pictures/5ffecc9b-d...,NaN,65.0
3,3,2,"{TV,""Cable TV"",Internet,""Wireless Internet"",""A...",1.0,Real Bed,1.0,1.0,strict,SF,t,...,-122.411906,BEST views + reviews! 5/5 stars*****,Nob Hill,38,Apartment,95.0,Private room,NaN,94133,166.0
4,4,2,"{TV,Internet,""Wireless Internet"",""Air conditio...",1.0,Real Bed,1.0,1.0,strict,NYC,t,...,-73.974691,Charming 1-bedroom - UWS Manhattan,Upper West Side,5,Apartment,100.0,Entire home/apt,https://a0.muscache.com/im/pictures/92879730/5...,10024,165.0


カテゴリデータを数値化しておく

In [337]:
# host_response_rate
train['host_response_rate'] = train['host_response_rate'].astype(str).apply(lambda x: x.replace('%', '')).astype(float)

# neighbourhood
le = LabelEncoder()
#ラベルを覚えさせる
le = le.fit(train['neighbourhood'].astype(str))
#ラベルを整数に変換
train['neighbourhood'] = le.transform(train['neighbourhood'].astype(str))

# property_type
le = LabelEncoder()
le = le.fit(train['property_type'].astype(str))
train['property_type'] = le.transform(train['property_type'].astype(str))

# zipcode
train['zipcode'] = train['zipcode'].astype(str).apply(lambda x : x[0:5].replace(' ', '0').replace('1m', '0').replace('Near', '0').replace('nan', '0') ).astype(int)


In [338]:
train = train[[#'id', 
               'accommodates',
               #'amenities', 
               'bathrooms', 
               'bed_type',
               'bedrooms',
               'beds', 
               'cancellation_policy', 
               'city',
               'cleaning_fee', 
               #'description',
               'first_review', 
               'host_has_profile_pic', 
               'host_identity_verified',
               'host_response_rate',
               'host_since', 
               'instant_bookable', 
               'last_review',
               'latitude', 
               'longitude', 
               #'name', 
               'neighbourhood',
               'number_of_reviews',
               'property_type', 
               'review_scores_rating', 
               'room_type', 
               #'thumbnail_url',
               'zipcode',
               'y']]

In [339]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 55583 entries, 0 to 55582
Data columns (total 24 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   accommodates            55583 non-null  int64  
 1   bathrooms               55436 non-null  float64
 2   bed_type                55583 non-null  object 
 3   bedrooms                55512 non-null  float64
 4   beds                    55487 non-null  float64
 5   cancellation_policy     55583 non-null  object 
 6   city                    55583 non-null  object 
 7   cleaning_fee            55583 non-null  object 
 8   first_review            43675 non-null  object 
 9   host_has_profile_pic    55435 non-null  object 
 10  host_identity_verified  55435 non-null  object 
 11  host_response_rate      41879 non-null  float64
 12  host_since              55435 non-null  object 
 13  instant_bookable        55583 non-null  object 
 14  last_review             43703 non-null

In [340]:
# train[['cancellation_policy', 'city', 'bed_type', 'cleaning_fee', 'host_has_profile_pic',
#       'host_identity_verified', 'instant_bookable', 'zipcode']].apply(lambda x: x.value_counts())

In [341]:
train_train = train.iloc[:50000,:]
train_test = train.iloc[50000:,:]

In [342]:
test =  pd.read_csv('../input/test.csv')
test.head()

,id,accommodates,amenities,bathrooms,bed_type,bedrooms,beds,cancellation_policy,city,cleaning_fee,...,latitude,longitude,name,neighbourhood,number_of_reviews,property_type,review_scores_rating,room_type,thumbnail_url,zipcode
0,0,6,"{TV,""Cable TV"",""Wireless Internet"",""Air condit...",2.0,Real Bed,2.0,2.0,strict,Boston,t,...,42.359278,-71.069962,Gorgeous 2BR/2BA Duplex in Beacon Hill,Beacon Hill,58,House,90.0,Entire home/apt,https://a0.muscache.com/im/pictures/7e4808b4-5...,02114
1,1,3,"{TV,""Cable TV"",Internet,""Wireless Internet"",""A...",1.0,Real Bed,1.0,1.0,moderate,LA,t,...,34.084747,-118.367355,Luxury 1 Bedroom West Hollywood City Center,West Hollywood,4,Guesthouse,100.0,Entire home/apt,https://a0.muscache.com/im/pictures/5392fbd6-6...,90046
2,2,2,"{TV,""Wireless Internet"",""Air conditioning"",Kit...",1.0,Real Bed,0.0,1.0,flexible,NYC,f,...,40.720541,-73.959192,Bedroom with Patio in Prime Williamsburg Locat...,Williamsburg,0,Apartment,NaN,Private room,https://a0.muscache.com/im/pictures/544d3b89-d...,11249.0
3,3,4,"{TV,""Cable TV"",Internet,""Wireless Internet"",""A...",1.0,Real Bed,1.0,2.0,strict,NYC,f,...,40.681117,-73.944091,Cozy apartment in Brooklyn,Bedford-Stuyvesant,0,Apartment,NaN,Entire home/apt,https://a0.muscache.com/im/pictures/26baf7ba-0...,11216
4,4,3,"{TV,Internet,""Wireless Internet"",""Air conditio...",1.5,Real Bed,1.0,2.0,strict,LA,t,...,34.150995,-118.409359,"Cozy, sunny, pet friendly loft/apt",NaN,6,Loft,92.0,Entire home/apt,https://a0.muscache.com/im/pictures/86107545/9...,91604


In [343]:
test =   test[[#'id', 
               'accommodates',
               #'amenities', 
               'bathrooms', 
               'bed_type',
               'bedrooms',
               'beds', 
               'cancellation_policy', 
               'city',
               'cleaning_fee', 
               #'description',
               'first_review', 
               'host_has_profile_pic', 
               'host_identity_verified',
               'host_response_rate',
               'host_since', 
               'instant_bookable', 
               'last_review',
               'latitude', 
               'longitude', 
               #'name', 
               'neighbourhood',
               'number_of_reviews',
               'property_type', 
               'review_scores_rating', 
               'room_type',
               #'thumbnail_url',
               'zipcode'
]]

In [344]:
# host_response_rate
test['host_response_rate'] = test['host_response_rate'].astype(str).apply(lambda x: x.replace('%', '')).astype(float)

# neighbourhood
le = LabelEncoder()
#ラベルを覚えさせる
le = le.fit(test['neighbourhood'].astype(str))
#ラベルを整数に変換
test['neighbourhood'] = le.transform(test['neighbourhood'].astype(str))

# property_type
le = LabelEncoder()
le = le.fit(test['property_type'].astype(str))
test['property_type'] = le.transform(test['property_type'].astype(str))

# zipcode
test['zipcode'] = test['zipcode'].astype(str).apply(lambda x : x[0:5].replace('nan', '0').replace('.', '0')).astype(int)

In [345]:
exp1 = setup(data = train_train, 
             test_data = train_test,
             target = 'y', 
             # ignore_features = None, 
             fold_shuffle= True, 
             #fold_strategy = tscv ,#'timeseries',
             data_split_shuffle = True)     

,Description,Value
0,session_id,2996
1,Target,y
2,Original Data,"(50000, 24)"
3,Missing Values,True
4,Numeric Features,9
5,Categorical Features,11
6,Ordinal Features,False
7,High Cardinality Features,False
8,High Cardinality Method,None
9,Transformed Train Set,"(50000, 144)"


In [ ]:
#best = compare_models(['lightgbm'])

In [346]:
best = create_model("lightgbm")

,MAE,MSE,RMSE,R2,RMSLE,MAPE
0,52.5609,11061.6706,105.1745,0.5928,0.4263,0.3780
1,52.4968,9561.4149,97.7825,0.6166,0.4192,0.3808
2,51.6338,11504.1561,107.2574,0.5811,0.4137,0.3621
3,53.5430,12204.8904,110.4757,0.5712,0.4145,0.3666
4,52.1770,11662.2115,107.9917,0.5713,0.4196,0.3738
5,51.3984,10678.4911,103.3368,0.6202,0.4116,0.3638
6,55.5367,13157.6068,114.7066,0.5451,0.4296,0.3803
7,53.4325,12355.6504,111.1560,0.5945,0.4264,0.3831
8,53.6840,13090.9722,114.4158,0.5932,0.4091,0.3519
9,52.2215,10525.4105,102.5934,0.6039,0.4107,0.3701


In [347]:
final_best = finalize_model(best)

In [348]:
prediction = predict_model(best, data=test)

In [350]:
prediction.to_csv("../output/prediction_add_categories.csv")